# Parameters for rate scans

In [ ]:
# --------- PARAMETERS --------- #

data_type    = 'l1'     # we can select for represent 'l0' or 'l1'

data_path    = ''       # input the path where we have the .result files copied from CaCo

# ------------------------------ #

# other parameters
plot_run     = True     # if we want or not to plot the individual graphs of each run and only full analysis
plot_camera  = True     # if we want to obtain the plots in the camera also 'lst environament' needed
data1run     = False    # giving the date '2022-04-21-12:15:47' for plotting only one run of all the set

plt_limit    = 2050     # the plot limits for the graphs (at 1000 we have the plateus)


# Reading the data

First we read the data of the runs/or the run

In [ ]:
import warnings
import numpy    as np

import extract  as ext  # this module extract the data
import graphs   as gr   # this module do the plots
import auxiliar as aux  # this module do the fitting, repair the pdfs and have the camera geometry change

#export all the data
ext.path_definition(data_path)
data, date, ranges, voltage, gain, dac, neigh = ext.select(data_type) 


#if we want to show only one run
if data1run != False:
    index = None                        #we check for the index that corresponds to wanted run
    for i in range(len(date)):
        if date[i] == data1run:
            index = i
            break
        if i == range(len(date))[-1]:
            print('ERROR, not found run in data, revise run name, input: "' + data1run + 
                  '" , expected format: yyyy-mm-dd-hh:min:sec')
    
    #we only get the data from the run we wanted to show
    voltage = [voltage[index]]
    ranges  = [ranges[ index]]
    neigh   = [neigh[  index]]
    data    = [data[   index]]
    date    = [date[   index]]
    gain    = [gain[   index]]
    dac     = [dac[    index]]

In [ ]:
#CONFIGURATION##############################################################

ext.create_folders()                          # creating the folders if they not exist
aux.parameters()                              # matplotlib parameters used for graphics
n,m,clusOrPx_name = ext.l0l1params(data_type) # parameters if we use l0 or l1, n,m = dim of multlti plots

warnings.filterwarnings("ignore")             # to avoid warnings at fitting points

try:
    gr.try_cta()                              # try if the cta environement is on
except:
    plot_camera = False                      # don't plot the camera if cta envrenement is not on
    print('ERROR: LST/CTA environement not active. Plots with the LST camera geometry will not be shown.'+
           'If wanted go to ctapipe github https://github.com/cta-observatory/ctapipe for more information.')
        
############################################################################

# Iterating all the runs

In [ ]:
# iteration over all runs, data[run]
print('Run  ' + str(0) + ' / ' + str(len(data))) # we print the run we are going trough to se the progress

# arrays for save data and complete analysis
meanARR,sigmaARR, treshold_devARR = [],[],[]

for run in range(len(data))[:]:

    # calculating times depending on the run, where ranges[run] = [start,end,step]
    t = np.linspace(ranges[run][0], ranges[run][1], int((ranges[run][1]-ranges[run][0])/ranges[run][2]+1))    
        
    
    # MULTIPLE PLOTTING IN DIFFERENT PAGES ·······························
    #####################################################################
    
    # multiple pages individual cluster/pixel plot
    # we will look if this is a valid run or not
    run_failed = False

    treshold   = gr.multiple_plot(t, data, run, ranges, m, n, clusOrPx_name, plt_limit, data_type, plot_run)

    NoneCount  = 0
    for i in range(len(treshold)):
        if treshold[i] == None:
            NoneCount = NoneCount + 1
            
    if NoneCount > (len(treshold) - 4):
        run_failed = True
        print('ERROR at run ' + date[run] + ', it\'s possible that data is empty (any '
              + clusOrPx_name + ' coud be fitted)')
        
    #####################################################################
    #····································································

    
    # HISTOGRAM ··························································
    #####################################################################
    
    # treshold treatment and gaussian fitting
    tresholdRaw,tresholdMean,tresholdZero,tresholdFitting,treshold_dev,Nzeros,mean,sigma = aux.treshold_tr(
        treshold, run_failed)
    
    meanARR.append(mean)
    sigmaARR.append(sigma)
    treshold_devARR.append(treshold_dev)
    
    # plot
    gr.histogram_treshold_plot(tresholdRaw, mean, sigma, Nzeros, run, gain, voltage, neigh, dac,
                               clusOrPx_name, run_failed, plot_run)
    
    #####################################################################
    #····································································
    
    
    

    # EACH CLUSTER TRESHOLD ··············································
    #####################################################################
    
    gr.cluster_treshold_plot(data, run, tresholdZero, mean, clusOrPx_name, gain, dac, voltage, neigh,
                             run_failed, plot_run)
           
    #####################################################################
    #····································································    
    
    
    
    
    # CAMERA REPRESENTATION ··············································
    #####################################################################

    if plot_camera == True:
        gr.camera_repr(data, tresholdMean, tresholdFitting, mean, Nzeros, run, gain, dac, neigh, voltage,
                       clusOrPx_name, data_type, run_failed, plot_run)
        
    #####################################################################
    #····································································
    
    
    
    
    # ALL CLUSTER/PIXELS TOGETHER ········································
    #####################################################################
    
    gr.all_together_plot(data, t, run, mean, plt_limit, gain, dac, neigh, voltage, clusOrPx_name,
                         run_failed, plot_run)
        
    #####################################################################
    #····································································
        
        
        
      
    # RUN PDF SAVE ·······················································
    #####################################################################
    
    # order of merged pdfs
    pdf_order = ['temp/histogram.pdf', 'temp/cluster_tresh.pdf', 'temp/camera_plot.pdf',
                 'temp/all_together.pdf', 'temp/multi_graphs.pdf']
    
    if plot_camera == False:
        pdf_order = [pdf_order[ii] for ii in range(len(pdf_order)) if pdf_order[ii] != 'temp/camera_plot.pdf']
        
    aux.merge_pdf(pdf_order, run, data_type, gain, date, neigh, voltage, dac,run_failed, 
                  plot_camera, plot_run)
    
    #####################################################################
    #····································································
    
    
    print('Run  ' + str(run + 1) + ' / ' + str(len(data)))   # we print the progress of the runs


# Complete analysis

With all the information we only will do the tabulation of the results for the runs without High Voltage because with HV we normally take only a few runs so we do not have statistics

In [ ]:
# COMPLETE ANALYSIS#######################################################

# NO HV·······································
# tresholds total plot ###################################################
gr.tresholds_total(meanARR, sigmaARR, gain, voltage, data_type, data1run, voltageName = 'no_HV')


# camera treshold deviation plot#########################################
gr.camera_total(treshold_devARR, data1run, plot_camera, clusOrPx_name, data_type)

# save pdf###############################################################
# order of merged pdfs
pdf_order = ['temp/treshold_total.pdf', 'temp/camera_total.pdf']

if plot_camera == False:
    pdf_order = [pdf_order[ii] for ii in range(len(pdf_order)) if pdf_order[ii] != 'temp/camera_total.pdf' ]

aux.merge_pdfTOT(pdf_order, data_type, plot_camera, plot_run, data1run, voltageName = 'no_HV')


'''
# HV ·······································
# tresholds total plot###################################################
gr.tresholds_total(meanARR, sigmaARR, gain, voltage, data_type, data1run, voltageName = 'HV')


# camera treshold deviation plot #########################################
gr.camera_total(treshold_devARR, data1run, plot_camera, clusOrPx_name, data_type)

# save pdf###############################################################
# order of merged pdfs
pdf_order=['temp/treshold_total.pdf','temp/camera_total.pdf']

if plot_camera == False:
    pdf_order = [pdf_order[ii] for ii in range(len(pdf_order)) if pdf_order[ii]!='temp/camera_total.pdf' ]

aux.merge_pdfTOT(pdf_order, data_type, plot_camera, plot_run, data1run, voltageName = 'HV')
'''
